In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from plotly import express as px

import ast
import pickle
from tqdm import tqdm

import networkx as nx
import random
import json

from collections import Counter

## Load in Data

### Load in Tweet Data

In [3]:
df = pd.read_csv('/Users/jchang153/Documents/UCLA-CAM/GV KG/Data/Tweets/All/bigbabies/bigbaby_r_t_30_s.csv')

In [4]:
# evaluating strings of lists as lists
df['hashtags'] = df['hashtags'].apply(ast.literal_eval)
df['mentions'] = df['mentions'].apply(ast.literal_eval)
# df['keywords'] = df['keywords'].apply(ast.literal_eval)

In [5]:
df.columns[:35]

Index(['created_at', 'tweet_id', 'text', 'in_reply_to_tweet_id',
       'in_reply_to_user_id', 'geo', 'quote_count', 'reply_count',
       'retweet_count', 'favorite_count', 'lang', 'quoted_tweet_id',
       'possibly_sensitive', 'user_id', 'screen_name', 'followers_count',
       'friends_count', 'statuses_count', 'verified', 'hashtags', 'mentions',
       'date', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11',
       '12'],
      dtype='object')

### Load in Crime Data

Load in the hate crime `hatecrimedata.csv` dataframe.

In [6]:
df_crime = pd.read_csv('/Users/jchang153/Documents/UCLA-CAM/GV KG/Data/hatecrimedata.csv')

In [7]:
# limiting to only dates that have associated tweets
df_crime['incident_date'] = pd.to_datetime(df_crime['incident_date'])
start_date = pd.to_datetime('2020-03-11')
end_date = pd.to_datetime('2021-06-17')

df_crime = df_crime[(df_crime['incident_date'] >= start_date) & (df_crime['incident_date'] <= end_date)]
df_crime['incident_date'] = pd.to_datetime(df_crime['incident_date']).dt.strftime('%Y-%m-%d')

In [8]:
df_crime = df_crime[df_crime['state_name'] == 'California']

In [9]:
df_crime.columns

Index(['incident_id', 'data_year', 'ori', 'pug_agency_name', 'pub_agency_unit',
       'agency_type_name', 'state_abbr', 'state_name', 'division_name',
       'region_name', 'population_group_code', 'population_group_description',
       'incident_date', 'adult_victim_count', 'juvenile_victim_count',
       'total_offender_count', 'adult_offender_count',
       'juvenile_offender_count', 'offender_race', 'offender_ethnicity',
       'victim_count', 'offense_name', 'total_individual_victims',
       'location_name', 'bias_desc', 'victim_types', 'multiple_offense',
       'multiple_bias'],
      dtype='object')

### Load in Topic Data

In [6]:
doc_info = pd.read_csv('/Users/jchang153/Documents/UCLA-CAM/GV KG/Data/Tweets/All/topics/bigbaby_r_30_docs.csv')
top_info = pd.read_csv('/Users/jchang153/Documents/UCLA-CAM/GV KG/Data/Tweets/All/topics/bigbaby_r_30_tops.csv')

In [7]:
top_info['Representation'] = top_info['Representation'].apply(ast.literal_eval)

In [8]:
top_info.columns

Index(['Topic', 'Count', 'Name', 'Representation', 'Representative_Docs'], dtype='object')

In [9]:
top2name = top_info.set_index('Topic')['Name'].to_dict()

### Querying by a Given Group

Only black-related topics:

In [10]:
# [32,35,65,135,142]
black_topics = [1,2,3,4,7,8,9,11,13,15,19,20,21,23,25,27,31,34,36,38,40,41,42,44,45,46,47,48,50,
51,52,56,62,63,64,66,68,70,71,73,75,76,78,82,83,84,86,88,92,93,94,101,102,105,106,
107,109,111,114,117,119,120, 121,122,123,124,125,126,131,132,133,137,140,143,144,145,
146,147,149,150,151,153,154,155,157,160,162,163,164,165,166,168,169,171,172,177,178,179,
180,184,186,187,188,191,193,196,200,202,203,206,208,211,212,216,218,220,222,223,224,225,
226,228,229,230,233,235,242,244,245,250,251,253,254,255,256,259,260,261,266,268,269]
lgbt_topics = [28,26,58,175,57,209,22,53,168,180,18,156,182,104,215,247]
jewish_topics = [79,24,72,0,30,37,55,87,91,129,121,129,199,205,264,10,59,5,6,100,262]
asian_topics = [67,69,96,115,135,147,161,16,99,183,195,201,221,243,101]
hispanic_topics = [32,35,65,135,142]

top_list = set(black_topics + lgbt_topics + asian_topics + jewish_topics + hispanic_topics)

In [11]:
groups = {('black', 'g'), ('lgbt', 'g'), ('asian', 'g'), ('jewish', 'g'), ('hispanic', 'g')}

In [12]:
len(top_list)

191

In [13]:
# topic2group = {\
# '78_blacklivesmatter_justiceforgeorgefloyd_blackouttuesday_blackheartredhandsfilm': [('black', 'g')],
# '166_blackouttuesday_lockdown_locked_theshowmustbepaused': [('black', 'g')],
# '181_black_tear_torn_men': [('black', 'g')],
# '341_africa_afrokixcollection_afropunk_african': [('black', 'g')],
# '560_naacp_african_image_awards': [('black', 'g')],
# '131_pride_gay_lgbtq_lgbt': [('lesbian', 'g'), ('gay', 'g'), ('bisexual', 'g'), ('transgender', 'g')],
# '740_saveartspace_futures_queer_selected': [('lesbian', 'g'),  ('gay', 'g'), ('bisexual', 'g'), ('transgender', 'g')],
# '632_jewcla_chabadlovesyou_chabadoncampus_jewishpride': [('jewish', 'g')],
# '262_floyd_george_lewis_floyds': [('black', 'g')],
# '396_breonna_taylor_taylors_cops': [('black', 'g')]
# }

topic2group = {}
for top in black_topics:
    if top in topic2group.keys():
        topic2group[top] = topic2group[top]+[('black', 'g')]
    else: 
        topic2group[top] = [('black', 'g')]

for top in lgbt_topics:
    if top in topic2group.keys():
        topic2group[top] = topic2group[top]+[('lgbt', 'g')]
    else: 
        topic2group[top] = [('lgbt', 'g')]
        
for top in asian_topics:
    if top in topic2group.keys():
        topic2group[top] = topic2group[top]+[('asian', 'g')]
    else: 
        topic2group[top] = [('asian', 'g')]
        
for top in jewish_topics:
    if top in topic2group.keys():
        topic2group[top] = topic2group[top]+[('jewish', 'g')]
    else: 
        topic2group[top] = [('jewish', 'g')]
        
for top in hispanic_topics:
    if top in topic2group.keys():
        topic2group[top] = topic2group[top]+[('hispanic', 'g')]
    else: 
        topic2group[top] = [('hispanic', 'g')]

Only tweets who have at least 0.01 probability of being in one of the above topics:

In [14]:
df = df[df[[str(num) for num in top_list]].ge(0.01).any(axis=1)]
len(df)

57158

In [13]:
df_crime['bias_desc'].unique()

NameError: name 'df_crime' is not defined

In [19]:
df_crime = df_crime[df_crime['bias_desc'].str.contains('|'.join(['black', 'asian', 'lesbian', 'transgender', 'gay', 'bisexual']), case=False)]

In [20]:
len(df_crime)

738

## Building the KG

In [15]:
# turns the dataframes into lists of dictionaries
tweet_dict = df.to_dict(orient='records')
# crime_dict = df_crime.to_dict(orient='records')
topic_dict = top_info.to_dict(orient='records')

### Getting unique entities, hashtags, topics

In [16]:
entities = set()
tags = set()
topics = set()
tweets = set()
users = set()
# crimes = set()
dates = set()

for tweet in tqdm(tweet_dict):
    entities.add((tweet['tweet_id'], 't'))
    # entities.add(tweet['screen_name']) # this one gives less entities, indicating some repeated screen names
    entities.add((tweet['user_id'], 'u'))
    entities.add(tweet['date'])
    # entities.add(tweet['topics'])
    
    tweets.add((tweet['tweet_id'], 't'))
    users.add((tweet['user_id'], 'u'))
    dates.add(tweet['date'])
    # topics.add(tweet['topics'])

    for tag in tweet['hashtags']:
        entities.add(tag)
        tags.add(tag)
        
# for crime in crime_dict:
#     entities.add((crime['incident_id'], 'c')) 
#     crimes.add((crime['incident_id'], 'c')) 
#     entities.add(crime['incident_date']) # in case not all are captured in tweets

for top in top_list:
    entities.add(top2name[top])
    topics.add(top2name[top])

entities = entities | groups
# entities.add('negative_sentiment')
# entities.add('positive_sentiment')
# entities.add('neutral_sentiment')

100%|███████████████████████████████████████████████████████████████████████████████████████| 57158/57158 [00:00<00:00, 200108.03it/s]


In [17]:
len(tweet_dict)

57158

In [18]:
# print(len(entities), len(tweets), len(users), len(crimes), len(dates), len(tags), len(topics), len(groups))
print(len(entities), len(tweets), len(users), len(dates), len(tags), len(topics), len(groups))

90815 57142 22065 464 10948 191 5


In [19]:
sum([51476, 21032, 738, 464, 10438, 230, 3])

84381

In [20]:
ent_list = list(entities)
entity2id = {k:v for v,k in enumerate(ent_list)}
id2entity = {v:k for v,k in enumerate(ent_list)}

### Creating relation dictionaries

In [28]:
# relations = ['tweeted_on', 'has_hashtag', 'in_topic', 'has_sentiment', 'tweeted', 'associated_with', 'occurred_on', 'mentioned', 'replied_to', 'victimized']
relations = ['tweeted_on', 'has_hashtag', 'in_topic', 'tweeted', 'mentioned', 'replied_to', 'associated_with']

relation2id = {}
id2relation = {}
j = 0 
for r in relations:
    relation2id[r] = j
    id2relation[j] = r
    j += 1

In [29]:
all_relations = []

## Mentions, Quotes, Replies Triples

### Exploring replies, quotes, mentions

Number of users that were replied to

In [24]:
all_replied_users = df.dropna(subset='in_reply_to_user_id')['in_reply_to_user_id'].tolist()

In [25]:
len(all_replied_users), len(set(all_replied_users))

(20543, 15790)

Number of users replied to in our dataset (not counting multiplicity, e.g., if a given user in our dataset was replied to multiple times, this would become multiple triples, but here we are just counting how many unique users that were replied to that are among the unique users in our dataset)

In [26]:
len(set(df['user_id']).intersection(set(all_replied_users)))

674

Number of tweets that were replied to

In [27]:
all_replied_tweets=df.dropna(subset='in_reply_to_tweet_id')['in_reply_to_tweet_id'].tolist()

In [28]:
len(all_replied_tweets), len(set(all_replied_tweets))

(1149, 1134)

Number of tweets replied to in our dataset (not counting multiplicity, e.g., if a given tweet in our dataset was replied to multiple times, this would become multiple triples, but here we are just counting how many unique tweets that were replied to that are among the unique tweets in our dataset)

In [29]:
len(set(df['tweet_id']).intersection(set(all_replied_tweets)))

44

Number of tweets that were quoted

In [30]:
len(df.dropna(subset='quoted_tweet_id'))

13003

Number of tweets quoted in our dataset

In [31]:
all_quoted_tweets=df.dropna(subset='quoted_tweet_id')['quoted_tweet_id'].explode().tolist()

In [32]:
len(set(df['tweet_id']).intersection(set(all_quoted_tweets)))

3

In [33]:
len(df[df['tweet_id'].isin(all_quoted_tweets)])

5

Number of users that were mentioned (counting multiplicity)

In [34]:
all_mentions = df[df["mentions"].apply(lambda x: len(x) > 0)]['mentions'].explode().tolist()

In [35]:
len(all_mentions), len(set(all_mentions))

(117814, 53935)

Number of mentioned users in our dataset

In [36]:
len(set(df['user_id']).intersection(set(all_mentions)))

4563

From the analysis above, we create only triples between tweets that reply to users and tweets that mention users.

### Tweet to User (reply) Triples

In [30]:
reply_relations = []

for tweet in tqdm(tweet_dict):
    replied_user = tweet['in_reply_to_user_id']
    
    if not np.isnan(replied_user):
        if (replied_user, 'u') in entities:

            relation = []
            relation.append(entity2id[(tweet['tweet_id'], 't')])
            relation.append(entity2id[(replied_user, 'u')])
            relation.append(relation2id['replied_to'])
            relation.append(np.nan)

            reply_relations.append(relation)

all_relations = [*all_relations, *reply_relations]

100%|███████████████████████████████████████████████████████████████████████████████████████| 57158/57158 [00:00<00:00, 177337.42it/s]


In [31]:
len(all_relations)

2533

### Tweet to User (mention) Triples

In [32]:
mention_relations = []

for tweet in tqdm(tweet_dict):  
    for men in tweet['mentions']:
        if (men, 'u') in entities:
            
            relation=[]
            relation.append(entity2id[(tweet['tweet_id'], 't')])
            relation.append(entity2id[(men, 'u')])
            relation.append(relation2id['mentioned'])
            relation.append(np.nan)
            
            mention_relations.append(relation)

all_relations = [*all_relations, *mention_relations]

100%|███████████████████████████████████████████████████████████████████████████████████████| 57158/57158 [00:00<00:00, 817557.23it/s]


In [33]:
len(mention_relations)

604

In [122]:
len(all_relations)

3137

## Tweet-to-? Triples

### Tweet-to-Hashtag Triples

In [34]:
tweet_hashtag_relations = []

for tweet in tqdm(tweet_dict):
    for tag in tweet['hashtags']:
        
        relation = []
        relation.append(entity2id[(tweet['tweet_id'], 't')])
        relation.append(entity2id[tag])
        relation.append(relation2id['has_hashtag'])
        relation.append(np.nan)
        
        tweet_hashtag_relations.append(relation)
        
all_relations = [*all_relations, *tweet_hashtag_relations]

100%|████████████████████████████████████████████████████████████████████████████████████████| 57158/57158 [00:00<00:00, 98571.18it/s]


In [35]:
len(all_relations)

58264

In [36]:
len(tweet_hashtag_relations)

55127

### Tweet-to-Date Triples

In [37]:
tweet_date_relations = []

for tweet in tqdm(tweet_dict):
        
    relation = []
    relation.append(entity2id[(tweet['tweet_id'], 't')])
    relation.append(entity2id[tweet['date']])
    relation.append(relation2id['tweeted_on'])
    relation.append(np.nan)
        
    tweet_date_relations.append(relation)

all_relations = [*all_relations, *tweet_date_relations]

100%|███████████████████████████████████████████████████████████████████████████████████████| 57158/57158 [00:00<00:00, 508431.21it/s]


In [39]:
len(tweet_date_relations)

57158

In [38]:
len(all_relations)

115422

### Tweet-to-Topic Triples

In [40]:
tweet_topic_relations = []

for tweet in tqdm(tweet_dict):
    for top in top_list:
        if tweet[f'{top}'] >= 0.01:
            # print(top)
            relation = []
            relation.append(entity2id[(tweet['tweet_id'], 't')])
            relation.append(entity2id[top2name[top]])
            relation.append(relation2id['in_topic'])
            relation.append(tweet[f'{top}'])
            
            tweet_topic_relations.append(relation)

all_relations = [*all_relations, *tweet_topic_relations]

100%|████████████████████████████████████████████████████████████████████████████████████████| 57158/57158 [00:02<00:00, 22525.17it/s]


In [41]:
len(tweet_topic_relations)

214821

In [42]:
len(all_relations)

330243

### Tweet-to-Sentiment Triples

In [ ]:
tweet_sentiment_relations = []

for tweet in tqdm(tweet_dict):
        
    relation = []
    relation.append(entity2id[(tweet['tweet_id'], 't')])
    sent = tweet['sentiment']
    
    if sent > 0:
        relation.append(entity2id['positive_sentiment'])
    elif sent == 0:
        relation.append(entity2id['neutral_sentiment'])
    else:
        relation.append(entity2id['negative_sentiment'])
    relation.append(relation2id['has_sentiment'])
    
    tweet_sentiment_relations.append(relation)

all_relations = [*all_relations, *tweet_sentiment_relations]

## Other Triples

### User-to-Tweet Triples

In [43]:
user_tweet_relations = []

for tweet in tqdm(tweet_dict):
        
    relation = []
    relation.append(entity2id[(tweet['user_id'], 'u')])
    relation.append(entity2id[(tweet['tweet_id'], 't')])
    relation.append(relation2id['tweeted'])
    relation.append(np.nan)

    user_tweet_relations.append(relation)
        
all_relations = [*all_relations, *user_tweet_relations]

100%|███████████████████████████████████████████████████████████████████████████████████████| 57158/57158 [00:00<00:00, 371407.01it/s]


In [44]:
len(user_tweet_relations)

57158

In [132]:
len(all_relations)

387401

### Topic-to-Group Triples

In [133]:
topic_group_relations = []

for topic in tqdm(topic_dict):
    if topic['Topic'] in topic2group.keys():
        for group in topic2group[topic['Topic']]:
            
            relation = []
            relation.append(entity2id[topic['Name']])
            relation.append(entity2id[group])
            relation.append(relation2id['associated_with']) 
            relation.append(np.nan)

            topic_group_relations.append(relation)
        
all_relations = [*all_relations, *topic_group_relations]

# topic_group_relations = []
# for topic in tqdm(black_topics):
    
#     num = ast.literal_eval(topic)
#     top = topic_info.iloc[num+1]['Name']
#     relation = []
#     relation.append(entity2id[top])
#     relation.append(entity2id[('black', 'g')])
#     relation.append(relation2id['associated_with'])
    
#     topic_group_relations.append(relation)

# all_relations = [*all_relations, *topic_group_relations]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 271/271 [00:00<00:00, 504731.96it/s]


In [134]:
len(topic_group_relations)

198

### Crime-to-Date Triples

In [59]:
crime_date_relations = []

for crime in tqdm(crime_dict):
    
    relation = []
    relation.append(entity2id[(crime['incident_id'], 'c')])
    relation.append(entity2id[crime['incident_date']])
    relation.append(relation2id['occurred_on'])
    
    crime_date_relations.append(relation)

all_relations = [*all_relations, *crime_date_relations]

100%|██████████████████████████████████████| 738/738 [00:00<00:00, 67323.42it/s]


In [60]:
len(crime_date_relations)

738

### Crime-to-Group Triples

In [61]:
crime_group_relations = []

for crime in tqdm(crime_dict):
    for group in groups:
        if group[0] in crime['bias_desc'].lower():
            
            relation = []
            relation.append(entity2id[(crime['incident_id'], 'c')])
            relation.append(entity2id[group])
            relation.append(relation2id['victimized'])
            
            crime_group_relations.append(relation)
    
all_relations = [*all_relations, *crime_group_relations]

100%|██████████████████████████████████████| 738/738 [00:00<00:00, 62495.38it/s]


In [68]:
for x in [x for x in all_relations if x[2] == relation2id['victimized']]

[[80647, 16597, 9],
 [56385, 16597, 9],
 [54613, 16597, 9],
 [65946, 16597, 9],
 [43312, 64110, 9],
 [47955, 64110, 9],
 [80200, 16597, 9],
 [27980, 16597, 9],
 [546, 64110, 9],
 [57549, 16597, 9],
 [83570, 16597, 9],
 [59277, 16597, 9],
 [62147, 64110, 9],
 [34758, 16597, 9],
 [68063, 16597, 9],
 [36429, 64110, 9],
 [43816, 64110, 9],
 [71727, 16597, 9],
 [22590, 16597, 9],
 [30670, 64110, 9],
 [4863, 16597, 9],
 [42051, 64110, 9],
 [17469, 16597, 9],
 [48437, 16597, 9],
 [76098, 16597, 9],
 [30185, 16597, 9],
 [84061, 64110, 9],
 [2643, 16597, 9],
 [7810, 16597, 9],
 [67691, 16597, 9],
 [12519, 64110, 9],
 [75158, 64110, 9],
 [66064, 16597, 9],
 [68950, 16597, 9],
 [15276, 16597, 9],
 [44447, 16597, 9],
 [75058, 16597, 9],
 [78052, 16597, 9],
 [17163, 16597, 9],
 [50664, 16597, 9],
 [53588, 64110, 9],
 [76811, 16597, 9],
 [18790, 64110, 9],
 [49471, 64110, 9],
 [78714, 16597, 9],
 [25004, 64110, 9],
 [30829, 16597, 9],
 [477, 16597, 9],
 [3319, 16597, 9],
 [26999, 16597, 9],
 [32487,

## Analysis

In [135]:
len(all_relations)

387599

Looking at numbers. crime-group is wrong bc it's not recognizing the lgbtq group...

In [65]:
len(reply_relations), len(mention_relations), len(tweet_hashtag_relations), len(tweet_date_relations), len(tweet_topic_relations), len(user_tweet_relations), len(crime_date_relations), len(crime_group_relations)

(2187, 545, 53275, 51491, 158057, 51491, 738, 502)

In [71]:
sum((2187, 545, 53275, 51491, 158057, 51491, 738, 738)) + 172

318694

Plus the length of topic to group triples

In [63]:
len(all_relations) + len(top_list)

318458

In [64]:
len(top_list)

172

In [136]:
all_relations2 = all_relations.copy()

In [137]:
for i in tqdm(all_relations2):
    i[0] = id2entity[i[0]]
    i[1] = id2entity[i[1]]
    i[2] = id2relation[i[2]]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 387599/387599 [00:00<00:00, 1625740.55it/s]


In [138]:
all_relations_df = pd.DataFrame(all_relations2, columns=['head', 'tail', 'relation', 'weight'])

In [139]:
all_relations_df

,head,tail,relation,weight
0,"(1.2378760749706363e+18, t)","(2251240369.0, u)",replied_to,NaN
1,"(1.2379704931702743e+18, t)","(7.566825919949169e+17, u)",replied_to,NaN
2,"(1.2379788759775887e+18, t)","(135319858.0, u)",replied_to,NaN
3,"(1.2386283599212708e+18, t)","(363269363.0, u)",replied_to,NaN
4,"(1.2390828352326902e+18, t)","(13753532.0, u)",replied_to,NaN
...,...,...,...,...
387594,262_hitler youth_youth_hitler_the hitler youth,"(jewish, g)",associated_with,NaN
387595,264_nazi rally_rally_1939_madison square,"(jewish, g)",associated_with,NaN
387596,266_blacklivesmatter theshowmustbepaused https...,"(black, g)",associated_with,NaN
387597,268_truth_georgefloyd_nojusticenopeace_georgef...,"(black, g)",associated_with,NaN


In [140]:
all_relations_df.to_csv('./data/kg_all_nocrimes.csv', index=False)

### Analyzing Graph

In [100]:
graph = nx.DiGraph()

# Add edges from the DataFrame
for _, row in all_relations_df.iterrows():
    graph.add_edge(row['head'], row['tail'], predicate=row['relation'])

# Draw the graph
# pos = nx.spring_layout(G, seed=42)  # Layout algorithm (you can try other layouts too)
# nx.draw(G, pos, with_labels=True, node_color='lightblue', font_size=12, node_size=2000, arrowsize=15)
# labels = nx.get_edge_attributes(G, 'predicate')
# nx.draw_networkx_edge_labels(G, pos, edge_labels=labels, font_size=10)

# plt.title("Triplet Data Graph")
# plt.show()

In [104]:
print(nx.is_weakly_connected(graph), nx.is_strongly_connected(graph))

True False


In [105]:
num_nodes = graph.number_of_nodes()
num_edges = graph.number_of_edges()

print(num_nodes, num_edges)

7638 14313


In [106]:
diameter = nx.diameter(graph)
print(diameter)

NetworkXError: Found infinite path length because the digraph is not strongly connected

In [107]:
clustering_coefficient = nx.average_clustering(graph)
print(clustering_coefficient)

0.0


In [108]:
degree_centrality = nx.degree_centrality(graph)
betweenness_centrality = nx.betweenness_centrality(graph)
closeness_centrality = nx.closeness_centrality(graph)

KeyboardInterrupt: 

In [111]:
list(nx.strongly_connected_components(graph))

7638

In [113]:
degree_centrality = nx.degree_centrality(graph)

# Get the most central node(s)
most_central_nodes = max(degree_centrality, key=degree_centrality.get)

print("Most central node(s) by degree centrality:", most_central_nodes)

Most central node(s) by degree centrality: 2_blacks_people_word_non


In [114]:
degree_centrality

{(1.2376865718364242e+18, 't'): 0.00026188293832656804,
 '2020-03-11': 0.0006547073458164201,
 (1.2377159175359734e+18, 't'): 0.00026188293832656804,
 (1.2377608926341898e+18, 't'): 0.00026188293832656804,
 (1.237764192116953e+18, 't'): 0.00026188293832656804,
 (1.237818232062407e+18, 't'): 0.00026188293832656804,
 (1.2379137078306245e+18, 't'): 0.00026188293832656804,
 '2020-03-12': 0.0011784732224695562,
 (1.2379166352115466e+18, 't'): 0.00026188293832656804,
 (1.2379447074826772e+18, 't'): 0.00026188293832656804,
 (1.2379788759775887e+18, 't'): 0.00026188293832656804,
 (1.2379802084031035e+18, 't'): 0.00026188293832656804,
 (1.2381052296539914e+18, 't'): 0.00026188293832656804,
 (1.2382028469164278e+18, 't'): 0.00026188293832656804,
 (1.2382547500906086e+18, 't'): 0.00026188293832656804,
 '2020-03-13': 0.0014403561607961243,
 (1.2382705550835098e+18, 't'): 0.00026188293832656804,
 (1.2383056563496755e+18, 't'): 0.00026188293832656804,
 (1.238323544003494e+18, 't'): 0.000261882938326